# Product Analytics Data Exploration

Interactive notebook for exploring the social media product analytics warehouse.

**Prerequisites:** Run `python run_pipeline.py` first to generate data and populate the warehouse.

In [ ]:
import sys
sys.path.insert(0, '..')

import duckdb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Connect to the warehouse
conn = duckdb.connect('../data/warehouse/product_analytics.duckdb', read_only=True)
print('Connected to warehouse')

# Quick overview
tables = conn.execute("""
    SELECT table_name, estimated_size 
    FROM duckdb_tables() 
    WHERE schema_name = 'analytics'
""").fetchdf()
print(tables)

## 1. DAU / WAU / MAU Trends

The most fundamental product metrics — tracking daily, weekly, and monthly active users across all platforms.

In [ ]:
# DAU trend with moving averages
dau_df = conn.execute("""
    SELECT
        date_key,
        SUM(dau) AS dau,
        SUM(total_events) AS total_events,
        SUM(total_sessions) AS total_sessions
    FROM analytics.agg_daily_metrics
    GROUP BY date_key
    ORDER BY date_key
""").fetchdf()

dau_df['dau_7d_ma'] = dau_df['dau'].rolling(7, min_periods=1).mean()
dau_df['events_per_dau'] = (dau_df['total_events'] / dau_df['dau']).round(1)

fig = go.Figure()
fig.add_trace(go.Bar(x=dau_df['date_key'], y=dau_df['dau'], name='DAU', opacity=0.5))
fig.add_trace(go.Scatter(x=dau_df['date_key'], y=dau_df['dau_7d_ma'], name='7-day MA', line=dict(width=3)))
fig.update_layout(title='Daily Active Users', template='plotly_dark', height=400)
fig.show()

## 2. Platform Performance Comparison

In [ ]:
# Platform-level metrics
platform_df = conn.execute("""
    SELECT
        m.platform_key,
        p.platform_name,
        ROUND(AVG(m.dau)) AS avg_dau,
        SUM(m.total_events) AS total_events,
        SUM(m.content_creates) AS content_creates,
        SUM(m.likes + m.comments + m.shares) AS interactions,
        SUM(m.ad_impressions) AS ad_impressions,
        SUM(m.ad_clicks) AS ad_clicks,
        ROUND(SUM(m.ad_clicks) * 100.0 / NULLIF(SUM(m.ad_impressions), 0), 3) AS ad_ctr
    FROM analytics.agg_daily_metrics m
    JOIN analytics.dim_platform p ON m.platform_key = p.platform_key
    GROUP BY m.platform_key, p.platform_name
    ORDER BY avg_dau DESC
""").fetchdf()

print(platform_df.to_string(index=False))

colors = {'facebook': '#1877F2', 'instagram': '#E4405F', 'messenger': '#00B2FF', 
          'whatsapp': '#25D366', 'threads': '#000000'}

fig = px.bar(platform_df, x='platform_name', y='avg_dau', 
             color='platform_key', color_discrete_map=colors,
             title='Average DAU by Platform')
fig.update_layout(template='plotly_dark', showlegend=False, height=400)
fig.show()

## 3. Engagement Funnel Analysis

Tracking the conversion funnel from passive viewing to active content creation.

In [ ]:
# Engagement funnel (latest date)
latest_date = conn.execute("SELECT MAX(date_key) FROM analytics.fct_events").fetchone()[0]

funnel_df = conn.execute(f"""
    SELECT
        COUNT(DISTINCT CASE WHEN event_type_key = 'content_view' THEN user_key END) AS viewers,
        COUNT(DISTINCT CASE WHEN event_type_key = 'like' THEN user_key END) AS likers,
        COUNT(DISTINCT CASE WHEN event_type_key = 'comment' THEN user_key END) AS commenters,
        COUNT(DISTINCT CASE WHEN event_type_key = 'share' THEN user_key END) AS sharers,
        COUNT(DISTINCT CASE WHEN event_type_key = 'content_create' THEN user_key END) AS creators
    FROM analytics.fct_events
    WHERE date_key = DATE '{latest_date}'
""").fetchone()

stages = ['View Content', 'Like', 'Comment', 'Share', 'Create Content']
values = list(funnel_df)

fig = go.Figure(go.Funnel(
    y=stages, x=values,
    textinfo='value+percent initial',
    marker=dict(color=['#1877F2', '#E4405F', '#25D366', '#00B2FF', '#FF6900'])
))
fig.update_layout(title=f'Engagement Funnel — {latest_date}', template='plotly_dark', height=400)
fig.show()

# Print conversion rates
print(f"View → Like:    {values[1]/values[0]*100:.1f}%")
print(f"View → Comment: {values[2]/values[0]*100:.1f}%")
print(f"View → Share:   {values[3]/values[0]*100:.1f}%")
print(f"View → Create:  {values[4]/values[0]*100:.1f}%")

## 4. Cross-Platform Usage Analysis

Understanding how users engage across the family of apps.

In [ ]:
cross_platform = conn.execute("""
    WITH user_platforms AS (
        SELECT user_key, COUNT(DISTINCT platform_key) AS platforms_used
        FROM analytics.fct_events
        GROUP BY user_key
    )
    SELECT
        platforms_used,
        COUNT(*) AS num_users,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 1) AS pct
    FROM user_platforms
    GROUP BY platforms_used
    ORDER BY platforms_used
""").fetchdf()

fig = px.bar(cross_platform, x='platforms_used', y='num_users',
             text='pct', title='Number of Platforms Used per User')
fig.update_traces(texttemplate='%{text}%', textposition='outside')
fig.update_layout(template='plotly_dark', height=400,
                  xaxis_title='Platforms Used', yaxis_title='Number of Users')
fig.show()

## 5. Power User Analysis (Pareto)

How concentrated is engagement among top users?

In [ ]:
power = conn.execute("""
    WITH user_events AS (
        SELECT user_key, COUNT(*) AS total_events,
               PERCENT_RANK() OVER (ORDER BY COUNT(*) DESC) AS pct_rank
        FROM analytics.fct_events
        GROUP BY user_key
    )
    SELECT
        CASE
            WHEN pct_rank <= 0.01 THEN 'Top 1%'
            WHEN pct_rank <= 0.05 THEN 'Top 5%'
            WHEN pct_rank <= 0.10 THEN 'Top 10%'
            WHEN pct_rank <= 0.20 THEN 'Top 20%'
            WHEN pct_rank <= 0.50 THEN 'Top 50%'
            ELSE 'Bottom 50%'
        END AS tier,
        COUNT(*) AS users,
        SUM(total_events) AS events,
        ROUND(SUM(total_events) * 100.0 / (SELECT SUM(total_events) FROM user_events), 1) AS pct_events
    FROM user_events
    GROUP BY tier
    ORDER BY MIN(pct_rank)
""").fetchdf()

print(power.to_string(index=False))

fig = px.bar(power, x='tier', y='pct_events', text='pct_events',
             title='Event Concentration by User Tier (Pareto Analysis)')
fig.update_traces(texttemplate='%{text}%', textposition='outside')
fig.update_layout(template='plotly_dark', height=400,
                  xaxis_title='User Tier', yaxis_title='% of Total Events')
fig.show()

## 6. User Retention Analysis

In [ ]:
from src.analytics.retention import RetentionAnalytics

ra = RetentionAnalytics(conn)

# D1, D3, D7 retention by user segment
for n in [1, 3, 7]:
    ret = ra.get_retention_by_segment(day_n=n)
    print(f"\n=== D{n} Retention by Segment ===")
    print(ret.to_string(index=False))

In [ ]:
# Churn risk analysis
churn = ra.get_churn_risk_features()

risk_dist = churn['churn_risk'].value_counts().reset_index()
risk_dist.columns = ['risk_level', 'count']

fig = px.pie(risk_dist, values='count', names='risk_level',
             title='User Churn Risk Distribution',
             color_discrete_sequence=['#25D366', '#FFD700', '#FF6900', '#E4405F'])
fig.update_layout(template='plotly_dark', height=400)
fig.show()

print(f"\nChurn Risk Summary:")
print(risk_dist.to_string(index=False))

## 7. Growth Accounting

Understanding DAU composition: new vs. retained vs. resurrected users.

In [ ]:
growth_df = conn.execute("""
    WITH daily_active AS (
        SELECT DISTINCT user_key, date_key
        FROM analytics.fct_events
    ),
    classified AS (
        SELECT
            curr.date_key,
            curr.user_key,
            CASE
                WHEN u.signup_date = curr.date_key THEN 'New'
                WHEN prev.user_key IS NOT NULL THEN 'Retained'
                ELSE 'Resurrected'
            END AS user_type
        FROM daily_active curr
        LEFT JOIN daily_active prev
            ON curr.user_key = prev.user_key
           AND prev.date_key = curr.date_key - INTERVAL '1 day'
        JOIN analytics.dim_users u
            ON curr.user_key = u.user_key AND u.is_current = TRUE
    )
    SELECT date_key, user_type, COUNT(DISTINCT user_key) AS users
    FROM classified
    GROUP BY date_key, user_type
    ORDER BY date_key
""").fetchdf()

fig = px.area(growth_df, x='date_key', y='users', color='user_type',
              color_discrete_map={'New': '#25D366', 'Retained': '#1877F2', 'Resurrected': '#FF6900'},
              title='Growth Accounting — DAU Composition')
fig.update_layout(template='plotly_dark', height=450)
fig.show()

## 8. Geographic & Demographic Insights

In [ ]:
# Geographic distribution
geo_df = conn.execute("""
    SELECT country, COUNT(DISTINCT user_key) AS users, COUNT(*) AS events
    FROM analytics.fct_events
    GROUP BY country
    ORDER BY users DESC
""").fetchdf()

fig = go.Figure(go.Choropleth(
    locations=geo_df['country'], z=geo_df['users'],
    locationmode='ISO-3', colorscale='Blues', colorbar_title='Users'
))
fig.update_layout(title='User Distribution by Country',
                  template='plotly_dark', height=450,
                  geo=dict(showframe=False, projection_type='natural earth'))
fig.show()

# Top 10 countries
print("Top 10 Countries by Users:")
print(geo_df.head(10).to_string(index=False))

In [ ]:
# Age group and device breakdown
demo_df = conn.execute(f"""
    SELECT
        u.age_group,
        u.device_type,
        COUNT(DISTINCT f.user_key) AS users,
        COUNT(*) AS events
    FROM analytics.fct_events f
    JOIN analytics.dim_users u ON f.user_key = u.user_key AND u.is_current = TRUE
    WHERE f.date_key = DATE '{latest_date}'
    GROUP BY u.age_group, u.device_type
""").fetchdf()

fig = px.treemap(demo_df, path=['age_group', 'device_type'], values='users',
                 title=f'User Distribution: Age Group × Device Type ({latest_date})')
fig.update_layout(template='plotly_dark', height=500)
fig.show()

## 9. Data Quality Summary

In [ ]:
from src.data_quality.checks import DataQualityChecker

dq = DataQualityChecker(conn)
results = dq.run_all_checks()

print(f"\nData Quality Report")
print(f"===================")
print(f"Total checks:  {results['total_checks']}")
print(f"Passed:        {results['passed']}")
print(f"Failed:        {results['failed']}")
print(f"Pass rate:     {results['pass_rate']}")

# Show any failures
failures = [d for d in results['details'] if d['status'] == 'failed']
if failures:
    print(f"\nFailed checks:")
    for f in failures:
        print(f"  - [{f['severity']}] {f['check']}: {f['message']}")
else:
    print("\nAll checks passed!")

In [ ]:
conn.close()
print('Warehouse connection closed.')